In [1]:
import nltk

In [2]:
from nltk import RegexpTokenizer

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\crisd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords

In [5]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup
import warnings


try:
    import nltk
except ModuleNotFoundError:
    !pip install nltk

try:
    import surprise
except ModuleNotFoundError:
    !pip install scikit-surprise
    import surprise
try:
    import scipy as sp
except ModuleNotFoundError:
    !pip install scipy
    import scipy as sp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize, RegexpTokenizer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
# Import
header = {
    'User-Agent': 'Chrome 108.0.5359.125',
    'Accept-Language': 'es'
}

In [7]:
class extraccionSinopsisPeliculas():
    def __init__(self):
        self.cargarFicheroSinopsisDataframe()
    
    def obtenerPeliculasSinId(self):
        self.df_links = pd.read_csv('csv/links.csv')
        self.df_links['tmdbId'] = df_links['tmdbId'].fillna(0)
        contador = []
        for i in range(len(df_links['tmdbId'])):
            if df_links["tmdbId"][i]==0:
                contador.append(df_links["movieId"][i])
                
        return contador
    
    
    def scrapingSinopsisPeliculas(self):
        columnaSinopsis = []
        for idPeli in df_links['tmdbId']:
            print(idPeli)
            if idPeli!=0:
                try:
                    url = "https://www.themoviedb.org/movie/"+str(int(idPeli))
                    page = requests.get(url,headers=header)
                    soup = BeautifulSoup (page.content, 'html.parser')
                    sinopsis = soup.find(class_="overview")
                    sinopsis = str(sinopsis.text)
                    columnaSinopsis.append(sinopsis)
                except:
                    columnaSinopsis.append("Sin Informacion")
            else:
                columnaSinopsis.append("Sin Informacion")

    def escribirSinopsisFichero(self):
        with open(r'sinopsis2.txt', "a", encoding="utf-8") as file:
            for i in columnaSinopsis:
                raw = repr(i)
                raw_replace = raw.replace('\\n', "").replace("'","")
                file.write(raw_replace+"|||")


    def cargarFicheroSinopsisDataframe(self):
        self.df_sinopsis = pd.DataFrame(columns=["sinopsis"])
        with  open('sinopsis2.txt', 'r', encoding="utf-8") as file:
            contents = file.read()
            contents = contents.split('|||')
            
        file.close()
        self.df_sinopsis["sinopsis"] =  contents
        self.df_sinopsis = self.df_sinopsis.iloc[:-1 , :]
        return self.df_sinopsis
    
    
    
    

extract = extraccionSinopsisPeliculas()  
df_sinopsisFinal = extract.cargarFicheroSinopsisDataframe()
df_movies = pd.read_csv('csv/movies.csv')
df_moviesSinopsis = pd.concat([df_movies, df_sinopsisFinal], axis=1)

In [8]:
df_peliculasConSinopsis = df_moviesSinopsis[df_moviesSinopsis["sinopsis"]!="Sin Informacion"]
df_peliculasConSinopsis = df_peliculasConSinopsis.reset_index()

In [9]:
class procesamientoTexto():
    
    # Funcion que pasa a minusculas y elimina los signos de puntuacion
    def tratamientoBasico(self, df_sinTratar):
        listatokens = []

        for indiceDF, fila in df_sinTratar.iterrows():
            tokenizer = RegexpTokenizer(r'\w+')
            tokens = tokenizer.tokenize(fila["sinopsis"])
            listatokens.append(tokens)

        for i in range(len(listatokens)):
            listatokens[i] = [w.lower() for w in listatokens[i]]
            df_sinTratar["sinopsis"][i] = listatokens[i]

        return df_sinTratar


    # Funcion que aplica las stopwords al datafram que se le pasa
    def quit_stopwords(self, df_conStopwords):
        listaStopwords = []
        stop_words_sp = set(stopwords.words('spanish'))
        try:
            filtered_sentence = []

            for indiceDF, fila in df_conStopwords.iterrows():
                filtered_sentence = [w for w in fila["sinopsis"] if not w in stop_words_sp]
                listaStopwords.append(filtered_sentence)

            for i in range(len(listaStopwords)):
                df_conStopwords["sinopsis"][i] = listaStopwords[i]

        except Exception as e:
            print(e)

        return df_conStopwords


    # Funcion que aplica el stemming al dataframe que se le pasa
    def stemming(self, df_sinStemming):
        listaStemming = []
        lista_stem = []

        # Se establece el idioma
        stemmer = SnowballStemmer('spanish')

        for indiceDF, fila in df_sinStemming.iterrows():
            if indiceDF != 0:
                lista_stem.append(listaStemming)
                listaStemming = []

            for word in range(len(fila["sinopsis"])):
                w = stemmer.stem(fila["sinopsis"][word])
                listaStemming.append(w)

        for i in range(len(lista_stem)):
            df_sinStemming["sinopsis"][i] = lista_stem[i]
        
        
        return df_sinStemming
    
    def prepararSinopsisTfidf(self, df_peliculasConSinopsis):
        listaUnidos = []
        for i in range(len(df_peliculasConSinopsis["sinopsis"])):
            unidos = " ".join(df_peliculasConSinopsis["sinopsis"][i])

            df_peliculasConSinopsis["sinopsis"][i] = str(unidos)
            
        return df_peliculasConSinopsis

procesar = procesamientoTexto()
procesar.tratamientoBasico(df_peliculasConSinopsis)
procesar.quit_stopwords(df_peliculasConSinopsis)
procesar.stemming(df_peliculasConSinopsis)
procesar.prepararSinopsisTfidf(df_peliculasConSinopsis)
df_peliculasConSinopsis.drop('index', inplace=True, axis=1)

<ipython-input-9-0520662388fa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinTratar["sinopsis"][i] = listatokens[i]
<ipython-input-9-0520662388fa>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conStopwords["sinopsis"][i] = listaStopwords[i]
<ipython-input-9-0520662388fa>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinStemming["sinopsis"][i] = lista_stem[i]
<ipython-input-9-0520662388fa>:67: SettingWithCopyWarn

In [10]:
class procesosSinopsis():


    def recommendations(self, titulo_pelicula, n_similares):
        # Se crea la instancia del tfidf
        tfidfvec = TfidfVectorizer()
        # Convierte el conjunto de datos en una matriz de tokens counts
        tfidf_movie = tfidfvec.fit_transform((df_peliculasConSinopsis["sinopsis"]))
        # Se obtiene la similitud del coseno
        cos_sim = cosine_similarity(tfidf_movie, tfidf_movie)
        # Se obtienen los indices de las peliculas
        indices = pd.Series(df_peliculasConSinopsis.index)

        recommended_movies = []
        # Se obtiene la pelicula pasada
        selected_movie = df_peliculasConSinopsis[df_peliculasConSinopsis["title"]==titulo_pelicula]
        # Se obtiene el indice de la pelicula
        selected_movie_index = selected_movie.index[0]
        # Se obtienen los puntuajes de las similitudes entre las peliculas ordenados de mayor a menor
        similarity_scores = pd.Series(cos_sim[selected_movie_index]).sort_values(ascending = False)
        # Se escogen el numero de peliculas especificadas
        numero_peliculas = list(similarity_scores.iloc[1:n_similares+1].index)
        print("Tu top", str(n_similares) + " de peliculas parecidas a " + titulo_pelicula +":")
        print("")
        contador = 1
        for i in numero_peliculas:
            # Se aniaden a la lista de peliculas recomendadas
            recommended_movies.append(list(df_peliculasConSinopsis.index)[i])
            print(str(contador)+ " - " + df_peliculasConSinopsis.loc[i]["title"])
            contador +=1
        #return recommended_movies

        

    def predecirRatingUsuarioSinopsis(self, user_id, titulo_pelicula):
        
        df_ratingsUsuarios = pd.read_csv('csv/ratings.csv')
    
        if len(df_ratingsUsuarios[df_ratingsUsuarios["userId"]==user_id])!=0:
            selected_user = df_ratingsUsuarios[df_ratingsUsuarios["userId"]==user_id]
            if len(df_peliculasConSinopsis[df_peliculasConSinopsis["title"]==titulo_pelicula])!=0:
                selected_movie = df_peliculasConSinopsis[df_peliculasConSinopsis["title"]==titulo_pelicula]
                selected_movieid = selected_movie["movieId"].values

                if len(selected_user[selected_user["movieId"]==selected_movieid[0]])!=0:
                    df_ratingUsuario = selected_user[selected_user["movieId"]==selected_movieid[0]]
                    ratingPelicula = df_ratingUsuario["rating"].values
                    print("El rating del usuario " + str(user_id) + " para la pelicula " + titulo_pelicula + " es de: " + str(ratingPelicula[0]))
                else:
                    df_userRatings_movies = pd.merge(selected_user, df_peliculasConSinopsis, on="movieId")
                    df_userRatings_movies = df_userRatings_movies.append(selected_movie, ignore_index=True)
                    # Se crea la instancia del tfidf
                    tfidfvec = TfidfVectorizer()
                    # Convierte el conjunto de datos en una matriz de tokens counts
                    tfidf_movie = tfidfvec.fit_transform((df_userRatings_movies["sinopsis"]))
                    # Se obtiene la similitud del coseno
                    cos_sim = cosine_similarity(tfidf_movie, tfidf_movie)


                    recommended_movies = []
                    selected_movie = df_userRatings_movies[df_userRatings_movies["title"]==titulo_pelicula]

                    # Se obtienen los puntuajes de las similitudes entre las peliculas ordenados de mayor a menor
                    similarity_scores = pd.Series(cos_sim[selected_movie_index]).sort_values(ascending = False)

                    # Se obtienen los indices de las peliculas que se parecen, y que no tienen un 0 de similitud
                    indices = []
                    for i in similarity_scores.index:
                        if similarity_scores.loc[i] != 0:
                            indices.append(i)

                    # Se obtienen las peliculas con tal indice
                    df_calculateRating = df_userRatings_movies.iloc[indices[1:]]
                    # Se calcula la media de todas ellas
                    prediction = format(df_calculateRating['rating'].mean(), '.3f')
                    print("")
                    print("El rating del usuario " + str(user_id) + " para la pelicula " + titulo_pelicula + " es de: " + str(prediction))
            else:
                print("La pelicula que desea predecir no contiene una sinopsis en que basarse")
        else:
            print("El usuario no existe")


        
proceso = procesosSinopsis()
proceso.recommendations("Wonder Woman (2009)", 5)
proceso.predecirRatingUsuarioSinopsis(1, "Jumanji (1995)")

Tu top 5 de peliculas parecidas a Wonder Woman (2009):

1 - Wonder Woman (2017)
2 - Justice League (2017)
3 - LEGO DC Super Hero Girls: Brain Drain (2017)
4 - DC Super Hero Girls: Hero of the Year (2016)
5 - Legend (1985)


NameError: name 'selected_movie_index' is not defined

In [51]:

class Procesos:
    def __init__(self):
        self.cargaDocumentos()
    def cargaDocumentos(self):
        self.df_links = pd.read_csv('csv/links.csv')
        self.df_links = self.df_links.dropna()
        self.df_movies = pd.read_csv('csv/movies.csv')
        self.df_movies = self.df_movies.dropna()
        self.df_ratings = pd.read_csv('csv/ratings.csv')
        self.df_ratings = self.df_ratings.dropna()
        self.df_tags = pd.read_csv('csv/tags.csv')
        self.df_tags = self.df_tags.dropna()
        self.df_movies_ratings = self.df_ratings.merge(self.df_movies)[['userId','movieId','title', 'rating','genres']]
        
        self.df_movies_ratings_tags = pd.merge(self.df_movies_ratings, self.df_tags, how='outer')[['userId','movieId','title', 'rating','genres', 'tag']]
        self.df_movies_ratings_tags["tag"] = self.df_movies_ratings_tags["tag"].str.lower()
        #self.df_movies_ratings_tags.fillna("vacio", inplace = True)
        
        self.ratings_table = self.df_movies_ratings.pivot_table(index='userId', columns='title', values='rating')
        #para cambiar los NAN por 0:
        self.ratings_table.fillna(0, inplace=True)
    def recomedacionPorValoracionOtrosUsuarios(self, nombrePelicula, n_similares):
        sparse_rating = sp.sparse.csr_matrix(self.ratings_table)
        similitud_usuarios = cosine_similarity(sparse_rating)
        #se hace con la transpuesta de la matriz creada anteriormente
        similitud_movies = cosine_similarity(sparse_rating.T)
        df_similitud_usuarios = pd.DataFrame(similitud_usuarios, index=self.ratings_table.index, columns=self.ratings_table.index)
        df_similitud_movies = pd.DataFrame(similitud_movies, index=self.ratings_table.columns, columns=self.ratings_table.columns)
        #le sumamos uno a n_similares porque la primera siempre es la propia pelicula y nos la saltamos
        n_similares+=1
        contador = 1
        print('Peliculas similares a ' + nombrePelicula + ':')
        print()
        for movie in df_similitud_movies[nombrePelicula].sort_values(ascending=False).index[1:n_similares]:
            print(str(contador) + ' - ' + str(movie))
            contador+=1
    def recomedacionPorGenero(self, nombrePelicula, n_similares):
        genres = list(set([genre for genres in self.df_movies["genres"].str.split("|") for genre in genres]))
        genre_matrix = []
        for index, row in self.df_movies.iterrows():
            genre_list = row["genres"].split("|")
            genre_vector = [1 if genre in genre_list else 0 for genre in genres]
            genre_matrix.append(genre_vector)
        genre_matrix = pd.DataFrame(genre_matrix, columns=genres)
        contador = 1
        selected_movie = self.df_movies[self.df_movies["title"] == nombrePelicula]
        selected_movie_index = selected_movie.index[0]
        #sacamos las similitudes de los generos
        similarities = cosine_similarity(genre_matrix[selected_movie_index:selected_movie_index+1], genre_matrix).flatten()
        #las metemos en una tupla y las ordenamos de mayor a menor 
        movie_list = [(index, similarity) for index, similarity in enumerate(similarities)]
        movie_list.sort(key=lambda x: x[1], reverse=True)

        print('Peliculas similares a ' + nombrePelicula + ':')
        print()
        #la bandera nos sirve para saltarnos la propia peli que buscamos
        #siempre esta a false y si nos encontramos la peli que estamos buscando la activamos a True
        #si esta en True al finalizar el bucle significa que ha saltado el titulo que buscabamos para no repetirse a si mismo 
        #y por lo tanto hay que añadir uno mas para llegar al numero deseado por el usuario
        bandera=False
        for movie in movie_list[0:n_similares]:
            if(nombrePelicula != self.df_movies.iloc[movie[0]]["title"]):
                print(str(contador)+' - ' +self.df_movies.iloc[movie[0]]["title"])
                contador+=1
            else:
                bandera=True
        if(bandera):
            #print('bandera')
            mov=movie_list[n_similares][0]
            print(str(contador)+' - ' +self.df_movies.iloc[mov]["title"])
    def predecirRatingDeUserAPeliculaPorSusGeneros(self, nombrePelicula, user_id):
        yaVotado = self.df_movies_ratings[(self.df_movies_ratings['title']==nombrePelicula) & (self.df_movies_ratings['userId']==user_id)]["rating"].unique()
        if(len(yaVotado)!=0):
            prediction = yaVotado[0]
            #print()
            print("La prediccion para " + nombrePelicula+" es: " + str(prediction))
            #return prediction
        else:
            # obtener géneros de la película a predecir
            movie_genres = self.df_movies_ratings[self.df_movies_ratings['title']==nombrePelicula]["genres"].unique()
            generosPeli = movie_genres[0].split("|")
            # filtrar valoraciones del usuario para peliculas con generos en comun
            user_ratings_ID = self.df_movies_ratings[self.df_movies_ratings['userId'] == user_id]
            user_ratings = user_ratings_ID.loc[user_ratings_ID['genres'].str.split('|').apply(lambda x: any(i in x for i in generosPeli))]
            # calcular la media de valoraciones del usuario para las peliculas con generos en comun
            if user_ratings.empty:
                print("La lista es empty")
                #return None
            else:
                #prediction = user_ratings_ID['rating'].mean()
                prediction = format(user_ratings['rating'].mean(), '.3f')
                #print()
                print("La prediccion para " + nombrePelicula + " es: " + str(prediction))
                #return prediction
    def recomedacionPorTags(self, nombrePelicula, n_similares):
        count_matrix = self.df_movies_ratings_tags.pivot_table(index='movieId', columns='tag', values='userId')
        #count_matrix = self.df_movies_ratings_tags.pivot_table(index='movieId', columns='tag', values='rating')
        count_matrix.fillna(0, inplace=True)
        sparse_rating = sp.sparse.csr_matrix(count_matrix)
        #print(sparse_rating)    
        selected_movie = self.df_movies[self.df_movies["title"] == nombrePelicula]["movieId"].values[0]
        #print(selected_movie)

        #encontramos el id de la pelicula en la matriz
        selected_movie_index = count_matrix.index.get_loc(selected_movie)

        similarities = cosine_similarity(sparse_rating, sparse_rating[selected_movie_index])

        movie_list = [(index, similarity) for index, similarity in enumerate(similarities)]
        movie_list.sort(key=lambda x: x[1], reverse=True)
        
        print('Peliculas similares a ' + nombrePelicula + ':')
        print()
        bandera=False
        contador = 1
        for movie in movie_list[0:n_similares]:
            if(nombrePelicula != self.df_movies.iloc[movie[0]]["title"]):
                print(str(contador)+' - ' +self.df_movies.iloc[movie[0]]["title"])
                contador+=1
            else:
                bandera=True
        if(bandera):
            mov=movie_list[n_similares][0]
            print(str(contador)+' - ' +self.df_movies.iloc[mov]["title"])
            
    def predecirRatingDeUserAPeliculaPorSusTags(self, nombrePelicula, user_id):
        yaVotado = self.df_movies_ratings[(self.df_movies_ratings['title']==nombrePelicula) & (self.df_movies_ratings['userId']==user_id)]["rating"].unique()
        if(len(yaVotado)!=0):
            prediction = yaVotado[0]
            #print()
            print("La prediccion para " + nombrePelicula+" es: " + str(prediction))
            return prediction
        else:
            # obtener tags de la película a predecir
            tagsPeli = []
            movie_tags = self.df_movies_ratings_tags[self.df_movies_ratings_tags['title']==nombrePelicula]["tag"].unique()
            for m in movie_tags:
                tagsPeli.append(m)
            #print(tagsPeli)
            filtroMergeandoTags = self.df_movies_ratings_tags[['userId','movieId','title', 'rating', 'tag']]
            filtroEnBaseUserId = filtroMergeandoTags[filtroMergeandoTags['userId']==user_id]
            
            #similitud = [distance.cosine(tagsPeli, j['tag']) for i,j in filtroEnBaseUserId.iterrows()]
            
            user_ratings = filtroEnBaseUserId[filtroEnBaseUserId['tag'].isin(tagsPeli)]
            #si el usuario a creado un tag de alguna peli que sea igual a alguno de el de la pelicula buscada filtramos mas el df quitando los nulos
            #si no a hecho ningun tag y todos sus tag son nan dejamos el df como esta ya que si hacemos dropna eliminamos el df entero
            if user_ratings.dropna().size != 0:
                user_ratings = user_ratings.dropna()
            user_ratings
            # calcular la media de valoraciones del usuario para las peliculas con generos en comun
            if user_ratings.empty:
                print("La lista es empty")
                return None
            else:
                #prediction = user_ratings_ID['rating'].mean()
                prediction = format(user_ratings['rating'].mean(), '.3f')
                #print()
                print("La prediccion para " + nombrePelicula + " es: " + str(prediction))
                return prediction

    def recomendacionEnBaseGeneroPelisQueNoHaVistoUsuario(self, user_id, n_similares):
        #warnings.filterwarnings('ignore')
        df_movies_rating_user = self.df_movies_ratings[self.df_movies_ratings['userId']==user_id]
        df_movies_rating_user = df_movies_rating_user.sort_values(by='rating',ascending=False)

        #cogemos los primeros 10 para ver que generos le gustan mas, anteriormente hemos ordenado por genero
        genero_mejor_rating_unicos = list(set([genre for genres in df_movies_rating_user.head(10)["genres"].str.split("|") for genre in genres]))

        # creamos un diccionario para guardar los generos y cuantas veces se repiten
        genre_count = {}
        for g in genero_mejor_rating_unicos:
            genre_count[g] = df_movies_rating_user.head(10)['genres'].str.count(g).sum()

        #ordenamos el diccionario de mayor a menor
        genero_mejor_rating = dict(sorted(genre_count.items(), key=lambda x: x[1], reverse=True))

        #sacamos las pelis que el usuario no ha visto
        df_movies_no_rating_user = self.df_movies[self.df_movies['movieId'].isin(df_movies_rating_user['movieId']) == False]
        #creamos en el df una columna por cada genero que le gusta al usuario y le agregamos cuanto le gusta
        for genre, weight in genero_mejor_rating.items():
            df_movies_no_rating_user[genre] = df_movies_no_rating_user["genres"].str.contains(genre).apply(lambda x: weight if x else 0)
        #creamos una nueva columna con la suma de cada fila para saber que peliculas le pueden gustar mas
        df_movies_no_rating_user["sumaPesos"] = df_movies_no_rating_user[genero_mejor_rating.keys()].sum(axis=1)
        #ordenamos por las pelis que tengan una mayor puntuacion en la columna sumaPesos ya que esto quiere decir que hay muchos generos que le gustan al usuario
        df_movies_no_rating_user = df_movies_no_rating_user.sort_values(by='sumaPesos',ascending=False)

        df_peliculas_mostrar = df_movies_no_rating_user['title'][0:n_similares]

        contador = 1
        for movie in df_peliculas_mostrar:
            print(str(contador)+' - ' +movie)
            contador+=1

        

            

p = Procesos()

#Extra
p.recomedacionPorValoracionOtrosUsuarios("Star Wars: Episode IV - A New Hope (1977)", 10)

print()
#Recomendaciones en base a caracteristicas sueltas
p.recomedacionPorGenero("Star Wars: Episode IV - A New Hope (1977)", 10)
print()
p.recomedacionPorTags("Star Wars: Episode IV - A New Hope (1977)", 10)
print()

#Prediccion de rating mediante un usuario dado
p.predecirRatingDeUserAPeliculaPorSusGeneros("Star Wars: Episode IV - A New Hope (1977)", 1)
print()
p.predecirRatingDeUserAPeliculaPorSusTags("City of God (Cidade de Deus) (2002)", 1)
print()

#Recomendacion en base a un usuario y caracteristicas
p.recomendacionEnBaseGeneroPelisQueNoHaVistoUsuario(1, 10)


Peliculas similares a Star Wars: Episode IV - A New Hope (1977):

1 - Star Wars: Episode V - The Empire Strikes Back (1980)
2 - Star Wars: Episode VI - Return of the Jedi (1983)
3 - Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
4 - Matrix, The (1999)
5 - Indiana Jones and the Last Crusade (1989)
6 - Back to the Future (1985)
7 - Star Wars: Episode I - The Phantom Menace (1999)
8 - Terminator, The (1984)
9 - Godfather, The (1972)
10 - Saving Private Ryan (1998)

Peliculas similares a Star Wars: Episode IV - A New Hope (1977):

1 - Waterworld (1995)
2 - Stargate (1994)
3 - Demolition Man (1993)
4 - Star Wars: Episode V - The Empire Strikes Back (1980)
5 - Star Wars: Episode VI - Return of the Jedi (1983)
6 - Star Trek III: The Search for Spock (1984)
7 - Lost in Space (1998)
8 - Rocketeer, The (1991)
9 - Tron (1982)
10 - Six-String Samurai (1998)

Peliculas similares a Star Wars: Episode IV - A New Hope (1977):

1 - Misérables, Les (1995)
2 - Double Happi

<ipython-input-51-aa4ec8b8b70d>:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_no_rating_user[genre] = df_movies_no_rating_user["genres"].str.contains(genre).apply(lambda x: weight if x else 0)
<ipython-input-51-aa4ec8b8b70d>:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_no_rating_user["sumaPesos"] = df_movies_no_rating_user[genero_mejor_rating.keys()].sum(axis=1)


In [52]:
user_id = 62
df_movies_rating_user = p.df_movies_ratings[p.df_movies_ratings['userId']==user_id]

In [53]:
df_movies_rating_user

,userId,movieId,title,rating,genres
278,62,6,Heat (1995),4.5,Action|Crime|Thriller
390,62,47,Seven (a.k.a. Se7en) (1995),4.5,Mystery|Thriller
878,62,110,Braveheart (1995),4.5,Action|Drama|War
1590,62,260,Star Wars: Episode IV - A New Hope (1977),4.5,Action|Adventure|Sci-Fi
1851,62,296,Pulp Fiction (1994),4.5,Comedy|Crime|Drama|Thriller
...,...,...,...,...,...
81526,62,182715,Annihilation (2018),4.0,Adventure|Mystery|Sci-Fi|Thriller
81529,62,183611,Game Night (2018),4.0,Action|Comedy|Crime|Horror
81530,62,184471,Tomb Raider (2018),3.5,Action|Adventure|Fantasy
81534,62,185031,Alpha (2018),4.5,Adventure|Thriller


In [54]:
p.df_movies[~p.df_movies.movieId.isin(df_movies_rating_user["movieId"])]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
6,7,Sabrina (1995),Comedy|Romance
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [55]:
df = pd.DataFrame()
movies = p.df_movies[~p.df_movies.movieId.isin(df_movies_rating_user["movieId"])]
df["movieId"] = movies["movieId"]
df["title"] = movies["title"]

In [57]:
df["ratingPredict"] = [p.predecirRatingDeUserAPeliculaPorSusTags(j["title"], user_id) for i,j in df.iterrows()]

La prediccion para Toy Story (1995) es: 3.000
La prediccion para Grumpier Old Men (1995) es: 4.131
La prediccion para Waiting to Exhale (1995) es: 4.131
La prediccion para Father of the Bride Part II (1995) es: 4.131
La prediccion para Sabrina (1995) es: 4.131
La prediccion para Tom and Huck (1995) es: 4.131
La prediccion para Sudden Death (1995) es: 4.131
La prediccion para GoldenEye (1995) es: 4.131
La prediccion para American President, The (1995) es: 3.500
La prediccion para Dracula: Dead and Loving It (1995) es: 4.131
La prediccion para Balto (1995) es: 4.131
La prediccion para Nixon (1995) es: 3.500
La prediccion para Cutthroat Island (1995) es: 4.131
La prediccion para Casino (1995) es: 5.000
La prediccion para Sense and Sensibility (1995) es: 4.131
La prediccion para Four Rooms (1995) es: 4.131
La prediccion para Ace Ventura: When Nature Calls (1995) es: 4.131
La prediccion para Money Train (1995) es: 4.131
La prediccion para Get Shorty (1995) es: 4.131
La prediccion para Copyc

In [58]:
df = df.sort_values(by='ratingPredict', ascending = False)

In [61]:
df.head(100)

,movieId,title,ratingPredict
1670,2247,Married to the Mob (1988),5.000
914,1213,Goodfellas (1990),5.000
6468,52712,"Invisible, The (2007)",5.000
6593,55290,Gone Baby Gone (2007),5.000
1124,1466,Donnie Brasco (1997),5.000
...,...,...,...
6436,51698,"Last Mimzy, The (2007)",4.131
6435,51694,Starter for 10 (2006),4.131
6467,52694,Mr. Bean's Holiday (2007),4.131
6434,51666,"Abandoned, The (2006)",4.131
